In [1]:
%pip install sae-lens transformer_lens sae-dashboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.1/145.1 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.6/160.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
from transformer_lens import HookedTransformer
from sae_lens import SAE
import torch
import plotly.express as px


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device : {device}")

Device : cpu


In [3]:
model_gpt2 = HookedTransformer.from_pretrained("gpt2-small", device=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


In [ ]:
logits = model_gpt2("Hello how are")

In [4]:
sae = SAE.from_pretrained(
    release="gpt2-small-res-jb",
    sae_id="blocks.8.hook_resid_pre",
    device=device,
)

sae.eval()

cfg.json: 0.00B [00:00, ?B/s]

blocks.8.hook_resid_pre/sae_weights.safe(…):   0%|          | 0.00/151M [00:00<?, ?B/s]

blocks.8.hook_resid_pre/sparsity.safeten(…):   0%|          | 0.00/98.4k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/sae_lens/saes/sae.py:248: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(


StandardSAE(
  (activation_fn): ReLU()
  (hook_sae_input): HookPoint()
  (hook_sae_acts_pre): HookPoint()
  (hook_sae_acts_post): HookPoint()
  (hook_sae_output): HookPoint()
  (hook_sae_recons): HookPoint()
  (hook_sae_error): HookPoint()
)

In [ ]:
from transformer_lens.utils import tokenize_and_concatenate

dataset = load_dataset(
    path="NeelNanda/pile-10k",
    split="train",
    streaming=False,
)

token_dataset = tokenize_and_concatenate(
    dataset=dataset,
    tokenizer=model_gpt2.tokenizer,
    streaming=True,
    max_length=sae.cfg.metadata.context_size,
    add_bos_token=sae.cfg.metadata.prepend_bos,
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (229134 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
inputs = token_dataset[:32]["tokens"]

with torch.no_grad():
    _, cache = model_gpt2.run_with_cache(inputs, prepend_bos=True)

In [ ]:
print(cache["blocks.8.hook_resid_pre"].shape)

torch.Size([32, 128, 768])


In [ ]:
with torch.no_grad():
  features_sae_encoded = sae.encode(cache[sae.cfg.metadata.hook_name])
  sae_decoded = sae.decode(features_sae_encoded)

In [ ]:
print(sae_decoded.shape)
print(features_sae_encoded.shape)

torch.Size([32, 128, 768])
torch.Size([32, 128, 24576])


In [ ]:
l0 = (features_sae_encoded[:, 1:] > 0).float().sum(-1).detach()
print("average l0", l0.mean().item())
px.histogram(l0.flatten().cpu().numpy()).show()

average l0 68.79060363769531


In [ ]:
from transformer_lens import utils
from functools import partial


# next we want to do a reconstruction test.
def reconstr_hook(activation, hook, sae_out):
    return sae_out


def zero_abl_hook(activation, hook):
    return torch.zeros_like(activation)


print("Orig", model_gpt2(inputs, return_type="loss").item())
print(
    "reconstr",
    model_gpt2.run_with_hooks(
        inputs,
        fwd_hooks=[
            (
                sae.cfg.metadata.hook_name,
                partial(reconstr_hook, sae_out=sae_decoded),
            )
        ],
        return_type="loss",
    ).item(),
)
print(
    "Zero",
    model_gpt2.run_with_hooks(
        inputs,
        return_type="loss",
        fwd_hooks=[(sae.cfg.metadata.hook_name, zero_abl_hook)],
    ).item(),
)

Orig 3.562199831008911
reconstr 3.764155387878418
Zero 11.146592140197754


In [ ]:
prompt = "When Bob and Mary went to the shop, Bob gave the bag to"
answer = "Mary"
utils.test_prompt(prompt, answer, model_gpt2, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'When', ' Bob', ' and', ' Mary', ' went', ' to', ' the', ' shop', ',', ' Bob', ' gave', ' the', ' bag', ' to']
Tokenized answer: [' Mary']


Performance on answer token:
Rank: 0        Logit: 17.42 Prob: 58.87% Token: | Mary|

Top 0th token. Logit: 17.42 Prob: 58.87% Token: | Mary|
Top 1th token. Logit: 15.36 Prob:  7.50% Token: | the|
Top 2th token. Logit: 14.80 Prob:  4.27% Token: | them|
Top 3th token. Logit: 14.75 Prob:  4.06% Token: | his|
Top 4th token. Logit: 14.28 Prob:  2.54% Token: | her|
Top 5th token. Logit: 14.23 Prob:  2.42% Token: | a|
Top 6th token. Logit: 13.85 Prob:  1.66% Token: | their|
Top 7th token. Logit: 13.37 Prob:  1.02% Token: | him|
Top 8th token. Logit: 13.09 Prob:  0.77% Token: | Mrs|
Top 9th token. Logit: 13.03 Prob:  0.73% Token: | Bob|


Ranks of the answer tokens: [(' Mary', 0)]

In [ ]:
tokens = model_gpt2.to_tokens(prompt)

logits, cache = model_gpt2.run_with_cache(prompt, prepend_bos=True)

hook_name = sae.cfg.metadata.hook_name

sae_out = sae(cache[hook_name])

orig_loss = model_gpt2(tokens, return_type="loss").item()
tweaked_loss = model_gpt2.run_with_hooks(tokens, return_type="loss", fwd_hooks=[(hook_name, partial(reconstr_hook, sae_out=sae_out))]).item()
zeros_loss = model_gpt2.run_with_hooks(tokens, return_type="loss", fwd_hooks=[(hook_name, zero_abl_hook)]).item()

In [ ]:
print(orig_loss)
print(tweaked_loss)
print(zeros_loss)

4.02357816696167
3.921599864959717
11.242877006530762


In [ ]:
with model_gpt2.hooks(fwd_hooks=[
    (hook_name, partial(reconstr_hook, sae_out=sae_out))
]):
    utils.test_prompt(prompt, answer, model_gpt2, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'When', ' Bob', ' and', ' Mary', ' went', ' to', ' the', ' shop', ',', ' Bob', ' gave', ' the', ' bag', ' to']
Tokenized answer: [' Mary']


Performance on answer token:
Rank: 0        Logit: 15.45 Prob: 16.95% Token: | Mary|

Top 0th token. Logit: 15.45 Prob: 16.95% Token: | Mary|
Top 1th token. Logit: 15.15 Prob: 12.60% Token: | her|
Top 2th token. Logit: 15.01 Prob: 11.01% Token: | them|
Top 3th token. Logit: 14.79 Prob:  8.78% Token: | the|
Top 4th token. Logit: 14.34 Prob:  5.61% Token: | Bob|
Top 5th token. Logit: 13.95 Prob:  3.79% Token: | a|
Top 6th token. Logit: 13.92 Prob:  3.68% Token: | their|
Top 7th token. Logit: 13.39 Prob:  2.18% Token: | him|
Top 8th token. Logit: 13.08 Prob:  1.59% Token: | his|
Top 9th token. Logit: 12.76 Prob:  1.16% Token: | me|


Ranks of the answer tokens: [(' Mary', 0)]

In [10]:
from sae_lens.analysis.neuronpedia_integration import get_neuronpedia_quick_list

In [ ]:
test_feature_idx_gpt = list(range(100))

neuronpedia_quick_list = get_neuronpedia_quick_list(sae, test_feature_idx_gpt)

print(neuronpedia_quick_list)

https://neuronpedia.org/quick-list/?name=temporary_list&features=%5B%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%220%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%221%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%222%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%223%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%224%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%225%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%226%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%227%22%7D%2C%20%7B%22modelId%22%3A%

In [ ]:
prompt = "I'd like to read a"

In [ ]:
print(generate_text(model_gpt2, prompt, 10))

I'd like to read a bit about the history of the "Pizza Hut


In [ ]:
_, cache = model_gpt2.run_with_cache(prompt)
with torch.no_grad():
  sae_encoded = sae.encode(cache[sae.cfg.metadata.hook_name])

In [ ]:
f26_direc = sae.W_dec[26]

In [ ]:
print(sae_encoded[sae_encoded != 0].mean())

tensor(17.3299, device='cuda:0')


In [ ]:
print(sae_encoded.shape)

torch.Size([1, 7, 24576])


In [ ]:
sae_encoded[0][-1][26] = 50
print(sae_encoded[0][-1][26])

tensor(50., device='cuda:0')


In [ ]:
decoded_sae = sae.decode(sae_encoded)

In [ ]:
from functools import partial

def reconstr_hook(activation, hook, sae_out):
    return sae_out

with model_gpt2.hooks(fwd_hooks=[
    (sae.cfg.metadata.hook_name, partial(reconstr_hook, sae_out=decoded_sae))
]):
    print(generate_text(model_gpt2, prompt, 10))

I'd like to read a little little little little little little little little little little


In [5]:
from functools import partial

In [6]:
def generate_text(model, sentence, tokens):
  for i in range(tokens):
    logits = model(sentence)
    maxed = torch.argmax(logits[0, -1])
    next = model.tokenizer.decode(maxed)
    sentence += next
  return sentence

def reconstr_hook_steer(activation, hook, sae_out, strenght):
    activation[:, -1, :] += sae_out * strenght
    return activation

def steer_output(model, sae, prompt, token, feature_to_steer, strengh):
  steering_vector = sae.W_dec[feature_to_steer]
  with model.hooks(fwd_hooks=[
    (sae.cfg.metadata.hook_name, partial(reconstr_hook_steer, sae_out=steering_vector, strenght = strengh))
]):
    print(generate_text(model, prompt, token))

In [ ]:
steer_output(model_gpt2, sae, "I work in a", 20, 34, 60)

I work in a small factory in a small town in the middle of the city. I have a small shop and a


In [56]:
# Find feature love
#prompt = "I love you so much, my heart if full of love."
prompt="I love you I love you so much"
print(model_gpt2.to_str_tokens(prompt, prepend_bos=False))
#prompt = "Love"
_, cache = model_gpt2.run_with_cache(prompt, prepend_bos=False)
love_pre = cache[sae.cfg.metadata.hook_name]
love_encoded = sae.encode(love_pre)
print(love_encoded.shape)

['I', ' love', ' you', ' I', ' love', ' you', ' so', ' much']
torch.Size([1, 8, 24576])


In [49]:
maxes = torch.max(love_encoded[0], dim=0)
print(maxes.values.shape)

torch.Size([24576])


In [60]:

maxes_indice = torch.topk(love_encoded[0][-4], 10).indices
print(maxes_indice)
neuronpedia_quick_list = get_neuronpedia_quick_list(sae, maxes_indice.tolist())

print(neuronpedia_quick_list)

tensor([11978, 10666, 16087,  5590, 12457, 14635,  2212, 12918, 16850, 16269])
https://neuronpedia.org/quick-list/?name=temporary_list&features=%5B%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%2211978%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%2210666%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%2216087%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%225590%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%2212457%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%2214635%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%222212%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2

In [64]:
steer_output(model_gpt2, sae, "I want to", 20, 11978, 60)

I want to make this a post about the game and the game itself. I want to make this a post about


In [84]:
print(sae.W_dec.shape)
word = "love"

token_love = model_gpt2.tokenizer.encode(word)
love_embedding = model_gpt2.W_E[token_love][0]

res = sae.W_dec @ love_embedding
print(res.shape)

maxes_indice = torch.topk(res, 10).indices
print(maxes_indice)
neuronpedia_quick_list = get_neuronpedia_quick_list(sae, maxes_indice.tolist())

print(neuronpedia_quick_list)


torch.Size([24576, 768])
torch.Size([24576])
tensor([10565, 21659, 20431, 18637, 14046,   501,  1520,  8954, 20184, 18525])
https://neuronpedia.org/quick-list/?name=temporary_list&features=%5B%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%2210565%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%2221659%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%2220431%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%2218637%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%2214046%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%22501%22%7D%2C%20%7B%22modelId%22%3A%20%22gpt2-small%22%2C%20%22layer%22%3A%20%228-res-jb%22%2C%20%22index%22%3A%20%221